In [ ]:
import random
from random import choice
import numpy as np
import time
import torch
import torch as T
device = T.device("cuda") 

In [ ]:
is16 = False

In [ ]:
class FinalEmbedding:
    def __init__(self,x,y):
        self.x_data = x
        self.y_data = y
      
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        preds = self.x_data[idx]
        trgts = self.y_data[idx] 
        sample = { 
        'predictors' : preds,
        'targets' : trgts
        }
        return sample

In [ ]:
from iopath.common.file_io import PathManager
from iopath.fb.manifold import ManifoldPathHandler

def load_checkpoint(checkpoint_path, map_location=None):
    pm = PathManager()
    pm.register_handler(ManifoldPathHandler())
    with pm.open(checkpoint_path, "rb") as f:
        if map_location is not None:
            checkpoint = torch.load(f, map_location=map_location)
        else:
            checkpoint = torch.load(f, map_location=lambda storage, loc: storage)
    return checkpoint


In [ ]:
api2indx = torch.load('gen1_33/api2indx.pt')
api2indx

In [ ]:
class pycoder_parameters:

    ''' Core Fuzzing Parameters '''
    NUM_FUZZ_PER_API= 100001 #000
    NUM_TEST_FUZZ = 2
    FLOAT_TENSOR = False #We either generate float or integer tensors
    UNIT_TEST = False
    COMPOSITE = True

    ''' Fuzzing Detailed Parameters '''
    MAX_TENSOR_DIMENSIONS = 3 #how many rows, columns, etc.
    MIN_VAL_PER_DIMENSION = 1 # e.g., min number of rows, columns, etc. 
    MAX_VAL_PER_DIMENSION = 5 # e.g., max number of rows, columns, etc. 

    #So far limiting to integers
    MIN_TENSOR_VALUE = 1
    MAX_TENSOR_VALUE = 15
    

    ''' Embedding Parameters '''
    EMBEDDING_NOISE_LEVEL = 0 #0 noise by default
    EMBEDDING_SIZE = 150
    SHAPE_EMBEDDING_SIZE = 6


    data_type = 'float' if FLOAT_TENSOR is  True else 'integer'
    model_type = 'Composite_' if COMPOSITE is  True else 'Single_'
    file_name = str(model_type) + str(NUM_FUZZ_PER_API) + '_' + data_type
    fuzzing   = file_name + '.pt'
    embedding = file_name + '.embedding' + '.pt',
    classification = file_name + '.model_result' + '.pt' 
    train_valid_test = file_name + 'train_valid_test.pt'

    def setNoiseLevel(self, noise):
        self.EMBEDDING_NOISE_LEVEL = noise
        self.embedding = self.file_name + '.embedding' + '_' + str(self.EMBEDDING_NOISE_LEVEL) + '.pt'

    def getEmbeddingFile(self):
        return(self.file_name + '.embedding' + '_' + str(self.EMBEDDING_NOISE_LEVEL) + '.pt')

    def getVisulizationFile(self):
        return(self.file_name + '.embedding' + '_' + str(self.EMBEDDING_NOISE_LEVEL) + '_' +  'tSNE.pt')

In [ ]:
NOISE = 0
f = pycoder_parameters()
f.setNoiseLevel(NOISE)
f.embedding = f.getEmbeddingFile() 
print(f.embedding)
print(f.SHAPE_EMBEDDING_SIZE)

In [ ]:
import sklearn.datasets
import torch
import numpy as np
import torch.nn.functional as F

EMBEDDING_SIZE = f.EMBEDDING_SIZE
SHAPE_EMBEDDING_SIZE = f.SHAPE_EMBEDDING_SIZE

def encode_values_to_code(tensor):
    tensor = tensor.clone()
    tensor[(tensor>=100) & (tensor<1000)] = 100
    tensor[(tensor>=1000)] = 101
    tensor[(tensor<=-20) & (tensor>-100)] = -20
    tensor[(tensor<=-100) & (tensor>-1000)] = -21
    tensor[(tensor<=-1000)] = -22
    return tensor

def tensor_flatten_pad(tensor, embed_size=EMBEDDING_SIZE, shape_embed_size=SHAPE_EMBEDDING_SIZE, isNoise=False):
    
    t_flatten = torch.flatten(tensor)

    padding_length = embed_size - list(t_flatten.shape)[-1]
    p1d = (0,padding_length) #just padding the last dimension
    t_pad = F.pad(input=t_flatten, pad=p1d, mode='constant', value=0).type(torch.FloatTensor)
    
    type_padding = 0
    if tensor.dtype == torch.bool:
        type_padding = 1
    elif tensor.dtype == torch.float64 \
        or tensor.dtype == torch.double \
        or tensor.dtype == torch.float32 \
        or tensor.dtype == torch.float16:
            type_padding = 2

    
    
    '''size embedding'''
    if(shape_embed_size > 0):
        t_shape = list(tensor.shape)
        padding_length = shape_embed_size -1 - len(t_shape)
        p1d = (0,padding_length) #just padding the last dimension
        s_pad = F.pad(input=torch.Tensor(t_shape), pad=p1d, mode='constant', value=0).type(torch.float)

        t_pad_list = t_pad.tolist()
        s_pad_list = s_pad.tolist()
        tensor_embedding = torch.Tensor([type_padding] + [-1] + t_pad_list + [-1] + s_pad_list + [-1])
    
    else:
        t_pad_list = t_pad.tolist()
        tensor_embedding = torch.Tensor([type_padding] + [-1] + t_pad_list + [-1])
        
    encoded_tensor = encode_values_to_code(tensor_embedding)
    return(encoded_tensor)

'unit test'
t1 = torch.tensor([[[1,2,3]],[[4,555,6]]])
tf = tensor_flatten_pad(t1, shape_embed_size=5, isNoise=True)
print(tf, tf.shape)

t2 = torch.tensor([[[True,False,True]],[[True,True,False]]])
tf1 = tensor_flatten_pad(t2, shape_embed_size=0)
print(tf1, tf1.shape)

In [ ]:
def embed_tensors(data_list):
       
    global EMBEDDING_SIZE
    global api2indx
    
    X=[]
    y=[]

    all_input_output_tensors = set()
    duplicate_count = 0

    dict_indx = len(api2indx)

    final_output = data_list[-1][1]

    prev_out = torch.Tensor()
    
    api_seq_x = []
    api_seq_y = []

    for data in data_list:
        api = data[0]      
        api_indx = api2indx[api]
        
        input_list = data[2] #.get_input()
        output_tensor = final_output #data.get_output()
        
        it_pad = []

        for input_tensor in input_list:
            if input_tensor.shape == prev_out.shape and torch.all(input_tensor.eq(prev_out)).item():
                t = torch.zeros(EMBEDDING_SIZE + SHAPE_EMBEDDING_SIZE + 1 + 2)
                t[-1] = -1
                it_pad.append(t)
            else:         
                #flatten the input tensor
                it_pad.append(tensor_flatten_pad(input_tensor,isNoise=True))
            
        
        #adding addidional tensors with zero embeddings for < 2 tensors
        for i in range(len(it_pad),3):
            t = torch.zeros(EMBEDDING_SIZE + SHAPE_EMBEDDING_SIZE + 1 + 2)
            t[-1] = -1
            it_pad.append(t)
       
        ot_pad = tensor_flatten_pad(output_tensor, isNoise=True)

        x = T.flatten(T.stack((it_pad[0],it_pad[1], it_pad[2], ot_pad)))


        xl = (x,api_indx)
        
        if xl in all_input_output_tensors:
            duplicate_count += 1
        else:
            all_input_output_tensors.add(xl)

    
        api_seq_x.append(x) 
        api_seq_y.append(api_indx)

        prev_out = data[1]

    X.append(api_seq_x)
    y.append(api_seq_y)
    
    return(X,y)


In [ ]:
import torch
from torch import nn

import numpy as np

In [ ]:
def process_dataX(tensor_list):
    io_seq = []
    
    n0 = tensor_list[0]

    if(len(tensor_list) == 1):
        n1 = torch.zeros(n0.shape)
        n2 = torch.zeros(n0.shape)
        
    elif(len(tensor_list) == 2):
        n1 = tensor_list[1]
        n2 = torch.zeros(n0.shape)

    elif(len(tensor_list) == 3):
        n1 = tensor_list[1]
        n2 = tensor_list[2]

        
    new_list = torch.stack((n0, n1, n2))
    io_seq.append(new_list)
    
    return(torch.stack(io_seq))

In [ ]:
indx2api = {}
EOS = '<eol>'

def process_dataY(api_seq):
    global indx2api
    global api2indx

    ''' Add <eol> to the dictionary '''
    indx2api = {v: k for k, v in api2indx.items()}

    if api2indx.get(EOS, -1) == -1:
        max_key = max(indx2api.keys())
        print(max_key)
        indx2api[max_key+1] = EOS
        api2indx[EOS] = max_key+1

    eos = api2indx[EOS]

    api_tensors = []


    api0 = api_seq[0]

    if len(api_seq) == 1:
        api1 = eos
        api2 = eos

    elif len(api_seq) == 2:
        api1 = api_seq[1]
        api2 = eos

    elif len(api_seq) == 3:
        api1 = api_seq[1]
        api2 = api_seq[2]

    else:
        print('!!! Not supposed to be here')

    t = torch.tensor([api0, api1, api2])
    api_tensors.append(t)
    
    return(torch.stack(api_tensors))

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNNModel, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True, bidirectional=True)   
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim*2, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out1 = out.contiguous().view(-1, self.hidden_dim*2)
        out1 = self.fc(out1)
        
        return out1, hidden, out
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers*2, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [ ]:
class FFNet(T.nn.Module):
  def __init__(self):
    super(FFNet, self).__init__()

    self.hid1 = T.nn.Linear(4*(f.EMBEDDING_SIZE+f.SHAPE_EMBEDDING_SIZE+1+2), 500)
    self.hid2 = T.nn.Linear(500, 250)
    self.hid3 = T.nn.Linear(250, 100)
    self.oupt = T.nn.Linear(100, len(api2indx))

    T.nn.init.xavier_uniform_(self.hid1.weight)
    T.nn.init.zeros_(self.hid1.bias)
    T.nn.init.xavier_uniform_(self.hid2.weight)
    T.nn.init.zeros_(self.hid2.bias)
    T.nn.init.xavier_uniform_(self.oupt.weight)
    T.nn.init.zeros_(self.oupt.bias)

    T.nn.Dropout(p=0.2)


  def forward(self, x):
    z1 = T.tanh(self.hid1(x))
    z2 = T.tanh(self.hid2(z1))
    z3 = T.tanh(self.hid3(z2))
    z = self.oupt(z3)  # no softmax: CrossEntropyLoss() 
    return (z, z3, z2, z1)

In [ ]:
DEBUG = True

In [ ]:
def embed_tensor_for_model(domain_io):

    x, y = embed_tensors(domain_io)

    X = process_dataX(x[0])
    Y = process_dataY(y[0])
    ds = FinalEmbedding(X,Y)

    X = ds[0]['predictors'].to(device)
    Y = ds[0]['targets'].to(device)  # [0] [1] or [2]

    return(X,Y)

In [ ]:
def beam_search(top3_list):

    api_seq = []
    api_seq1 = []
    for i in top3_list[0]:
        for j in top3_list[1]:
            api_seq1.append(((i[0], j[0]), i[1]*j[1]))
    
    api_seq1.sort(key = lambda x: x[1], reverse=True) 

    for k in top3_list[2]:
        for s1 in api_seq1:
            api_seq.append(((s1[0][0], s1[0][1], k[0]), s1[1]*k[1]))

    api_seq.sort(key = lambda x: x[1], reverse=True) 
    
    return(api_seq)
            

In [ ]:
total_correct = {}
seq_len_correct = {}
seq_len_total = {}
total_benchmark = 0

In [ ]:
def reset_stat():
    global total_correct
    global seq_len_correct
    global seq_len_total
    global total_benchmark
    
    total_correct = {}
    seq_len_correct = {}
    seq_len_total = {}
    total_benchmark = 0

In [ ]:
def print_stat():
    print('total_correct', total_correct)
    print('seq_len_correct', seq_len_correct)
    print('seq_len_total', seq_len_total)
    print('total_benchmark', total_benchmark)

In [ ]:
def query_model(X, Y):
    
    global total_benchmark
    with T.no_grad():
      start_time = time.time()
      predicts, z3, z2, z1 = net(X)
      temp_z3 = torch.unsqueeze(z3,0)
      model_output, hidden, int_output = rnn_model(temp_z3)

      target_list = list(Y.cpu().numpy())

    top_indx = []

    top3_list = []

    print('---- Top-k ----')
    for i, m in enumerate(model_output):
        top3 = []
        prob = nn.functional.softmax(m, dim=0).data
        # Taking the class with the highest probability score from the output
        xx = torch.topk(prob,3,dim=0)[1]
        for x in xx.cpu().numpy():
          print(indx2api[x])
          top3.append((indx2api[x], prob[x].item()))
        top3_list.append(top3)
        print('====')
        
    print('---- Predicted ----')
    for m in model_output:
        prob = nn.functional.softmax(m, dim=0).data
        api_ind = torch.max(prob, dim=0)[1].item()
        top_indx.append(api_ind)
        if DEBUG:
            print(indx2api[api_ind], prob[api_ind])


    print('---- Expected ----')
    seq_len = 0
    for o in list(Y):
        print(o.item())
        if o.item() != len(api2indx)-1:
          seq_len += 1
        print(indx2api[o.item()])
    print('seq length', seq_len)
    if seq_len_correct.get(seq_len, -1) == -1:
      seq_len_correct[seq_len] = 0
      seq_len_total[seq_len] = 0
    
    seq_len_total[seq_len] += 1
    total_benchmark += 1

    o = list(Y)
    expected = ((indx2api[o[0].item()], indx2api[o[1].item()], indx2api[o[2].item()]))
    
    top3_list = beam_search(top3_list)
    for i,t in enumerate(top3_list):
      if t[0] == expected:
        print(i+1, t[0], '\t***MATCH')
        if total_correct.get(i+1, -1) == -1:
          total_correct[i+1] = 0
        total_correct[i+1] += 1
        seq_len_correct[seq_len] += 1
        break
      else:
        print(t[0])
    return(top3_list)


In [ ]:
def api_edit_distance(seq1, seq2):

    edit_distantce = 0

    for i in range(len(seq1)):
        if seq1[i] != seq2[i]:
            edit_distantce += 1

    return(edit_distantce)

In [ ]:
net = torch.load('net_model.pt')
rnn_model = torch.load('rnn_model.pt')

In [ ]:
reset_stat()

In [ ]:
print_stat()

In [ ]:
def Composite01():
    in1 = torch.tensor([[5, 2], [1, 3], [0, 2]])
    out_orig = torch.tensor([[[5, 5], [1, 1], [0, 0]],
                                [[2, 2], [3, 3], [2, 2]]])                            
    out = torch.transpose(in1.expand((2, 3, 2)), 0, 2)

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(in1, out))
    
    domain_io = []
    domain_io.append(['expand', torch.transpose(in1.expand((2, 3, 2)), 0, 2), [in1]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    print('-------------------------------------')

    domain_io = []
    domain_io.append(['transpose', torch.transpose(in1.expand((2, 3, 2)), 0, 2), [in1.expand((2, 3, 2))]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)
        
Composite01()

In [ ]:
def Composite02():
    domain_inpt10 = torch.tensor([5, 1, 0, 3, 0, 0, 2, 0, 2])
    domain_out10 = torch.lt(domain_inpt10, 1)
    
    domain_inpt20 = domain_out10
    domain_inpt21 = domain_inpt10

    domain_output_orig = torch.tensor([1, 1, 0, 1, 0, 0, 1, 0, 1])
    domain_output = torch.where(domain_inpt20, domain_inpt21, 1)

    print('domain_inpt10 : ', domain_inpt10, domain_inpt10.shape)
    print('domain_inpt20 : ', domain_inpt20, domain_inpt20.shape)
    print('domain_output : ', domain_output, domain_output.shape)
    print(torch.equal(domain_output, domain_output_orig))

    domain_io = []
    domain_io.append(['lt', domain_output, [domain_inpt10, torch.tensor(1)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    domain_io = []
    domain_io.append(['where', domain_output, [domain_inpt20, domain_inpt21, torch.tensor(1)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite02()

In [ ]:
print_stat()

In [ ]:
def Composite05():
    in1 = torch.tensor([[4, 3, 1], [6, 5, 2]])
    in2 = torch.tensor([[[5, 5]], [[1, 5]], [[6, 0]]])
    output_orig = torch.tensor([[[29, 35]], [[47, 55]]])
    output = torch.tensordot(in1, in2, 1)
    
    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(output, output.shape, output.dtype)
    print(torch.equal(output_orig, output))

    domain_io = []
    domain_io.append(['tensordot', output, [in1, in2]])
    
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite05()

In [ ]:
def Composite06():
    in1 = torch.tensor([3, 5, 0, 2, 3, 3, 0])
    in2 = torch.unsqueeze(in1, 1)
    output_orig = torch.tensor([
                [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0],
                [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
                [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
                [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0],
                [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0],
                [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
            ]).int()
    output = torch.eq(in1, in2).int()

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(output, output.shape, output.dtype)
    print(torch.equal(output, output_orig))

    domain_io = []
    domain_io.append(['unsqueeze',  torch.eq(in1, in2), [in1]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    domain_io = []
    domain_io.append(['eq', torch.eq(in1, in2), [in1, torch.unsqueeze(in1, 1)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite06()

In [ ]:
def Composite07():
    in1 = torch.tensor([
                    [[8, 4, 6], [2, 12, 3]],
                    [[11, 12, 5], [9, 12, 12]],
                    [[9, 2, 13], [7, 0, 7]],
                    [[2, 10, 5], [7, 1, 2]],
                ])
    output_orig = torch.tensor([
                [[8, 4, 6], [11, 12, 5], [9, 2, 13], [2, 10, 5]],
                [[2, 12, 3], [9, 12, 12], [7, 0, 7], [7, 1, 2]],
            ])
    output = torch.transpose(in1, 0, 1)

    print(in1, in1.shape, in1.dtype)
    print(output, output.shape, output.dtype)
    print(torch.equal(output, output_orig))

    domain_io = []
    domain_io.append(['transpose', output, [in1]])
    
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite07()

In [ ]:
def Composite08():
    in1 = torch.tensor([1, 0, 0, 2, 1, 3, 5, 0, 1, 2, 10])
    in2 = torch.tensor([12, 3, 45, 6, 7, 8, 9, 87, 65, 4, 32])
    in3 = torch.gt(in1, 1)
    output_orig = torch.tensor([6, 8, 9, 4, 32])
    output = torch.masked_select(in2, torch.gt(in1, 1))

    print('in1: ', in1, in1.shape, in1.dtype)
    print('in2: ', in2, in2.shape, in2.dtype)
    print('in3: ', in2, in2.shape, in2.dtype)
    print('output', output, output.shape, output.dtype)
    print(torch.equal(output, output_orig))

    domain_io = []   
    domain_io.append(['gt', torch.masked_select(in2, torch.gt(in1, 1)), [in1, torch.tensor(1)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    domain_io = []   
    domain_io.append(['masked_select', torch.masked_select(in2, torch.gt(in1, 1)), [in2, torch.gt(in1, 1)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite08()

In [ ]:
def Composite11():
    in1 = torch.tensor([4, 0, 1, 1, 0, 4, 0, 0, 3, 4, 1])
    out_orig = torch.tensor([4, 3, 0, 1, 3])
    out = torch.bincount(in1)

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['bincount', torch.bincount(in1), [in1]])
    
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite11()

In [ ]:
def Composite13():
    in1 = torch.tensor([[3, 5], [10, 2]])
    in2 = torch.tensor([[[1, 0], [5, 4]], [[3, 10], [2, 0]]])
    in3 = torch.matmul(in1, in2)
    out_orig = torch.tensor([[[28, 20], [19, 30]], [[20, 8], [34, 100]]])
    out = torch.transpose(torch.matmul(in1, in2), 0, 1)

    print('in1 : ', in1, in1.shape, in1.dtype)
    print('in2 : ', in2, in2.shape, in2.dtype)
    print('in3 : ', in3, in3.shape, in3.dtype)
    print('output : ', out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['matmul', torch.transpose(torch.matmul(in1, in2), 0, 1), [in1, in2]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    domain_io = []
    domain_io.append(['transpose', torch.transpose(torch.matmul(in1, in2), 0, 1), [torch.matmul(in1, in2)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite13()

In [ ]:
def Composite14():
    in1 = torch.tensor([
                    [
                        [False, False, True],
                        [False, False, False],
                        [True, False, True],
                        [False, True, False],
                        [False, False, False],
                        [True, True, True],
                        [True, True, False],
                    ]
                ]).int()
    out_orig = torch.tensor([[True, False, True, True, False, True, True]]).int()
    out = torch.any(in1, -1).int()

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['any', torch.any(in1, -1), [in1]])
    
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite14()

In [ ]:
def Composite15():
    in1 = torch.tensor([3, 1, 2, 0, 1, 0, 10, 1, 0])
    out_orig = torch.tensor([3, 0, 2, 0, 0, 0, 10, 0, 0])
    out = torch.mul(in1, torch.ne(in1, 1)) 

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['ne', torch.mul(in1, torch.ne(in1, 1)), [in1, torch.tensor(1)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


    domain_io = []
    domain_io.append(['mul', torch.mul(in1, torch.ne(in1, 1)), [in1, torch.ne(in1, 1)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite15()

In [ ]:
def Composite16():
    in1 = torch.tensor([[2, 5], [3, 0], [8, 7]])
    in2 = torch.tensor([4, 10, 6])
    out_orig = torch.tensor([[8, 20], [30, 0], [48, 42]])
    out = torch.mul(in1, torch.unsqueeze(in2, 1))

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = [] 
    domain_io.append(['unsqueeze',torch.mul(in1, torch.unsqueeze(in2, 1)), [in2]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


    domain_io = [] 
    domain_io.append(['mul', torch.mul(in1, torch.unsqueeze(in2, 1)), [in1,torch.unsqueeze(in2, 1)]])    
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite16()

In [ ]:
def Composite17():
    in1 = torch.tensor([17, 32, 99])
    out_orig = torch.tensor([[17, 17], [32, 32], [99, 99]])
    out = torch.stack((in1, in1), 1)

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []    
    domain_io.append(['stack', out, [in1, in1]])
    
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite17()

In [ ]:
def Composite18():
    in1 = torch.tensor([[[1, 1, 1], [1, 0, 1]], [[1, 2, 3], [4, 5, 6]]])
    in2 = torch.tensor([[1, 1, 1, 1], [1, 2, 3, 4], [5, 6, 7, 8]])
    in3 = torch.tensor([100, 200, 300, 400])
    in4 = torch.matmul(in1, in2)

    out_orig = torch.tensor([
                [[107, 209, 311, 413], [106, 207, 308, 409]],
                [[118, 223, 328, 433], [139, 250, 361, 472]],
            ])
    out = torch.add(in3, torch.matmul(in1, in2))

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(in3, in3.shape, in3.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['matmul', out, [in1, in2]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


    domain_io = []
    domain_io.append(['add', out, [in3, torch.matmul(in1, in2)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite18()

In [ ]:
# NOT SUPPORTING by 16
def Composite20():
    if is16 == True:
        return
    in1 = torch.tensor([
                    [7, 2, 1],
                    [4, 5, 1],
                    [4, 4, 2],
                    [3, 4, 3],
                    [0, 0, 1],
                ])
    out_orig = torch.tensor([[1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
    out = torch.nn.functional.one_hot(torch.argmax(in1, 1), 3)

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))


    domain_io = []
    domain_io.append(['argmax', torch.nn.functional.one_hot(torch.argmax(in1, 1),3), [in1]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    domain_io = []
    domain_io.append(['one_hot', torch.nn.functional.one_hot(torch.argmax(in1, 1),3), [torch.argmax(in1, 1), torch.tensor(3)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite20()

In [ ]:
# NOT SUPPORTED by 16
def Composite21():
    if is16 == True:
        return
    in1 = torch.tensor([[2], [0], [1], [0]])
    in2 = torch.tensor([[2, 5, 3], [1, 3, 6], [1, 6, 3], [7, 0, 3]])
    out_orig = torch.tensor([[3], [1], [6], [7]])
    out = torch.gather(in2, 1, in1)

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['gather', out, [in2, in1]])

    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


Composite21()

In [ ]:
def Composite22():
    in1 = torch.tensor([3, 1, 10])
    in2 = torch.tensor([[6, 4], [5, 1], [3, 4]])
    out_orig = torch.tensor([53, 53])
    out = torch.tensordot(in1, in2, 1 )

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['tensordot', out, [in1, in2]])


    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite22()

In [ ]:
# NOT SUPPORTED by 16
def Composite23():
    if is16 == True:
        return
    in1 = torch.tensor([[0, 5, 2], [3, 1, 4], [5, 1, 5]])
    out_orig = torch.tensor([
                [1, 0, 1, 0, 0, 1, 0, 0, 0],
                [0, 1, 0, 1, 1, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 1, 0, 0, 0],
            ])
    out = torch.max(torch.nn.functional.one_hot(in1, 9), 1)[0]

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['one_hot', torch.sum(torch.nn.functional.one_hot(in1, 9), 1), [in1, torch.tensor(9)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


    domain_io = []
    domain_io.append(['sum', torch.sum(torch.nn.functional.one_hot(in1, 9), 1), [torch.nn.functional.one_hot(in1, 9)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


Composite23()

In [ ]:
# NOT SUPPORTED by 16
def Composite24():
  if is16 == True:
        return
  in1 = torch.tensor([3, 1, 4, 5, 2, 8, 6, 7])
  in2 = 0
  in3 = torch.tensor([1, 0, 2, 0, 1, 1, 0, 2])
  out_orig = torch.tensor([3, 1, 2, 5, 2, 8, 6, 3.5])
  out = torch.where(torch.ne(in3, in2), torch.div(in1, in3), in1.float())

  print(in1, in1.shape, in1.dtype)
  print(in2)
  print(in3, in3.shape, in3.dtype)
  print(out, out.shape)
  print(torch.equal(out, out_orig))


  domain_io = []
  domain_io.append(['ne', torch.where(torch.ne(in3, in2), torch.div(in1, in3), in1.float()), [in3, torch.tensor(in2)]])
  X, Y = embed_tensor_for_model(domain_io)
  query_model(X, Y)

  domain_io = []
  domain_io.append(['div', torch.where(torch.ne(in3, in2), torch.div(in1, in3), in1.float()), [in1, in3]])
  X, Y = embed_tensor_for_model(domain_io)
  query_model(X, Y)


  domain_io = []
  domain_io.append(['where', torch.where(torch.ne(in3, in2), torch.div(in1, in3), in1.float()), [torch.ne(in3, in2), torch.div(in1, in3), in1]])
  X, Y = embed_tensor_for_model(domain_io)
  query_model(X, Y)

Composite24()

In [ ]:
# NOT SUPPORTED BY 16 APIS
def Composite25():
    if is16 == True:
        return
    out_orig = torch.tensor([
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1],
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1],
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1],
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1],
            ]).int()
    out = torch.tile(torch.eye(3), (4, 1)).int()

    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['eye', torch.tile(torch.eye(3), (4, 1)), []])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    domain_io = []
    domain_io.append(['tile', torch.tile(torch.eye(3), (4, 1)), [torch.eye(3)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite25()

In [ ]:
# NOT SUPPORTED BY 16 APIs
def Composite26():
    if is16 == True:
        return
    in1 = torch.tensor([[[3, 4], [1, 2]], [[5, 2], [10, 3]], [[10, 20], [4, 7]]])
    out_orig = torch.tensor([10, 20, 41])
    out = torch.sum(torch.sum(in1, 1), 1)

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['sum', torch.sum(torch.sum(in1, 1), 1), [in1]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


    domain_io = []
    domain_io.append(['sum', torch.sum(torch.sum(in1, 1), 1), [torch.sum(in1, 1)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite26()

In [ ]:
# NOT SUPPORTED BY 16 APIs
def Composite27():
    if is16 == True:
        return
    in1 = torch.tensor([0, 3, 5, 6])
    out_orig = torch.tensor([1, 0, 0, 1, 0, 1, 1, 0])
    out = torch.sum(torch.nn.functional.one_hot(in1, 8), 0)

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['one_hot', torch.sum(torch.nn.functional.one_hot(in1, 8), 0), [in1, torch.tensor(8)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    domain_io = []
    domain_io.append(['sum', torch.sum(torch.nn.functional.one_hot(in1, 8), 0), [torch.nn.functional.one_hot(in1, 8)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite27()

In [ ]:
# NOT SUPPORTED BY 16 APIs
def Composite29():
    if is16 == True:
        return
    in1 = torch.tensor([1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21])
    in2 = torch.tensor([12, 0, 10, 23, 16])
    out_orig = torch.tensor([6, 0, 5, 11, 8])
    out = torch.searchsorted(in1, in2)

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['searchsorted', out, [in1, in2]])

    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite29()

In [ ]:
# NOT SUPPORTED BY 16 APIs
import math
def Composite30():
    if is16 == True:
        return
    in1 = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    in2 = torch.tensor([[9.0, 4.0], [8.0, 5.0], [7.0, 6.0]])
    out_orig = torch.tensor([[math.sqrt(68), math.sqrt(58), math.sqrt(52)],
                  [math.sqrt(36), math.sqrt(26), math.sqrt(20)],
                  [math.sqrt(20), math.sqrt(10), math.sqrt(4)]])
    out = torch.cdist(in1, in2)

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []

    domain_io.append(['cdist', out, [in1, in2]])

    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite30()

In [ ]:
# NOT SUPPORTED BY 16 APIs
def Composite32():
    if is16 == True:
        return
    in1 = torch.tensor([[1, 6, 2, 1], [3, 1, 4, 2], [2, 1, 2, 5]])
    out_orig = torch.tensor([13, 15, 20])
    out = torch.tensordot(in1, torch.arange(4), 1)

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['arange', torch.tensordot(in1, torch.arange(4), 1), []])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


    domain_io = []
    domain_io.append(['tensordot', torch.tensordot(in1, torch.arange(4), 1), [in1, torch.arange(4)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


Composite32()

In [ ]:
def Composite34():
    in1 = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]], [[10, 20], [30, 40]]])
    in2 = torch.tensor([3, 5, 10])
    out_orig = torch.tensor([[128, 236], [344, 452]])
    out = torch.tensordot(in2, in1, 1)

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['tensordot', out, [in1, in2]])

    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite34()

In [ ]:
# NOT SUPPORTED BY 16 APIs
def Composite36():
    if is16 == True:
        return
    in1 = torch.tensor([1, 0, 1, 1, 0, 1, 0, 1])
    out_orig = torch.tensor([1.0, 0.0, 0.333333, 0.25, 0.0, 0.166667, 0.0, 0.125])
    out = torch.div(in1, torch.add(in1, torch.arange(8)))

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(out_orig, out_orig.shape, out_orig.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['arange', torch.arange(8), []])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)
    print('--------------------------------------')

    domain_io = []
    domain_io.append(['add', torch.div(in1, torch.add(in1, torch.arange(8))), [in1, torch.arange(8)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)
    print('--------------------------------------')

    domain_io = []
    domain_io.append(['div', torch.div(in1, torch.add(in1, torch.arange(8))), [in1, torch.add(in1, torch.arange(8))]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite36()

In [ ]:
def Composite37():
    in1 = torch.tensor([
                    [
                        [[10, 20, 30], [40, 50, 60]],
                        [[12, 34, 56], [78, 98, 76]],
                    ]
                ])
    in2 = torch.tensor([5, 10, 20])
    out_orig = torch.tensor([[[850, 1900], [1520, 2890]]])
    out = torch.tensordot(in1, in2, 1)

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['tensordot', out, [in1, in2]])

    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite37()

In [ ]:
# NOT SUPPORTED BY 16 APIs
def Composite39():
    if is16 == True:
        return
    in1 = torch.tensor([[15, 10, 9, 20], [11, 0, 1, 9], [10, 1, 11, 25]])
    out_orig = torch.tensor([
                [225, 100, 81, 400],
                [121, 0, 1, 81],
                [100, 1, 121, 625],
            ])
    out = torch.square(torch.mul(in1, in1.bool()))

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['mul', torch.square(torch.mul(in1, in1.bool().long())), [in1, in1.bool().long()]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    domain_io = []
    domain_io.append(['square', torch.square(torch.mul(in1, in1.bool().long())), [torch.mul(in1, in1.bool().long())]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite39()

In [ ]:
# NOT SUPPORTED BY 16 APIs
#torch.masked_select(in1, torch.ne(torch.arange(10), in2))
def Composite41():
    if is16 == True:
        return
    in1 = torch.tensor([5, 2, 8, 2, 4, 1, 1, 0, 2, 1])
    in2 = 3
    out_orig = torch.tensor([5, 2, 8, 4, 1, 1, 0, 2, 1])
    out = torch.masked_select(in1, torch.ne(torch.arange(10), in2))

    print(in1, in1.shape, in1.dtype)
    print(in2)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['arange', torch.masked_select(in1, torch.ne(torch.arange(10), in2)),[]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


    domain_io = []
    domain_io.append(['ne', torch.masked_select(in1, torch.ne(torch.arange(10), in2)), [torch.arange(10), torch.tensor(3)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


    domain_io = []
    domain_io.append(['masked_select',torch.masked_select(in1, torch.ne(torch.arange(10), in2)), [in1, torch.ne(torch.arange(10), in2)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


Composite41()

In [ ]:
def Composite42():
    in1 = torch.tensor([4, 6, 2, 6, 7, 3, 3])
    out_orig = torch.tensor([0, 0, 0, 0, 1, 0, 0]).int()
    out = torch.eq(in1, 7)

    domain_io = []

    domain_io.append(['eq', torch.eq(in1, 7), [in1, torch.tensor(7)]])

    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite42()

In [ ]:
# NOT SUPPORTED BY 16 APIs
def Composite44():
    if is16 == True:
        return
    in1 = torch.tensor([
                    [3, 5, 2],
                    [6, 2, 3],
                    [8, 7, 1],
                    [0, 3, 5],
                    [4, 7, 3],
                    [2, 1, 6],
                    [10, 20, 30],
                    [4, 5, 6],
                ])                
    out_orig = torch.tensor([[9, 7, 5], [8, 19, 6], [6, 8, 9], [14, 25, 36]])
    out = torch.sum(torch.reshape(in1, (-1, 2, 3)), 1)

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['reshape', torch.sum(torch.reshape(in1, (-1, 2, 3)), 1), [in1]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    print('-----------------------')

    domain_io = []
    domain_io.append(['sum', torch.sum(torch.reshape(in1, (-1, 2, 3)), 1), [torch.reshape(in1, (-1, 2, 2))]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite44()

In [ ]:
# NOT SUPPORTED BY 16 APIs
def Composite45():
  if is16 == True:
        return
  in1 = torch.tensor([1, 0, 1, 0, 1])
  in2 = torch.tensor([[[12, 34], [56, 78], [23, 54], [76, 78], [42, 24]]])
  out_orig = torch.tensor([[[34, 12],
         [56, 78],
         [54, 23],
         [76, 78],
         [24, 42]]])
  out = torch.where(torch.unsqueeze(in1, 1).bool(), torch.roll(in2, 1, -1), in2)

  print(in1, in1.shape)
  print(in2, in2.shape)
  print(out, out.shape)
  print(torch.equal(out, out_orig))
  
  domain_io = []
  domain_io.append(['unsqueeze',torch.where(torch.unsqueeze(in1, 1).bool(), torch.roll(in2, 1, -1), in2), [in1]])
  X, Y = embed_tensor_for_model(domain_io)
  query_model(X, Y)


  domain_io = []
  domain_io.append(['roll', torch.where(torch.unsqueeze(in1, 1).bool(), torch.roll(in2, 1, -1), in2), [in2]])
  X, Y = embed_tensor_for_model(domain_io)
  query_model(X, Y)

  domain_io = []
  domain_io.append(['where', torch.where(torch.unsqueeze(in1, 1).bool(), torch.roll(in2, 1, -1), in2), [torch.unsqueeze(in1, 1).bool(), torch.roll(in2, 1, -1), in2]])
  X, Y = embed_tensor_for_model(domain_io)
  query_model(X, Y)

Composite45()

In [ ]:
# NOT SUPPORTED BY 16 APIs
def Composite46():
    if is16 == True:
        return
    in1 = torch.tensor([3, 4, 1])
    out_orig = torch.tensor([0, 0, 0, 1, 1, 1, 1, 2])
    out = torch.repeat_interleave(torch.arange(3), in1, 0)

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['arange', torch.repeat_interleave(torch.arange(3), in1, 0), []])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


    domain_io = []
    domain_io.append(['repeat_interleave', torch.repeat_interleave(torch.arange(3), in1, 0), [torch.arange(3)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite46()

In [ ]:
# NOT SUPPORTED BY 16 APIs
def Composite48():
    if is16 == True:
        return
    in1 = torch.tensor([32, 53, 45, 38, 29, 89, 64, 23])
    in2 = torch.tensor([38, 53, 89, 38, 32, 64])
    out_orig = torch.tensor([3, 1, 5, 3, 0, 6])
    out = torch.argmax(torch.eq(in1, torch.unsqueeze(in2, 1)).int(), 1)

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    print('torch.unsqueeze(in1, 1)', torch.unsqueeze(in1, 1))
    print('torch.eq(in2, torch.unsqueeze(in1, 1)).float()', torch.eq(in2, torch.unsqueeze(in1, 1)).int())
    print('torch.argmax(torch.eq(in2, torch.unsqueeze(in1, 1)).float(), 1)', torch.argmax(torch.eq(in2, torch.unsqueeze(in1, 1)).int(), 1))

    domain_io = []
    domain_io.append(['unsqueeze',torch.argmax(torch.eq(in2, torch.unsqueeze(in1, 1)).float(), 1), [in1]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    domain_io = []
    domain_io.append(['eq', torch.argmax(torch.eq(in2, torch.unsqueeze(in1, 1)).float(), 1), [in2,torch.unsqueeze(in1, 1).float()]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    domain_io = []
    domain_io.append(['argmax', torch.argmax(torch.eq(in2, torch.unsqueeze(in1, 1)).float(), 1), [torch.eq(in2, torch.unsqueeze(in1, 1)).float()]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite48()

In [ ]:
def Composite49():
    in1 = torch.tensor([
                    [[[1, 2, 3], [4, 5, 6]]],
                    [[[8, 10, 0], [6, 4, 2]]],
                    [[[9, 8, 7], [1, 2, 3]]],
                ])
    in2 = torch.tensor([20, 5, 10])
    out_orig = torch.tensor([
                [[[20, 40, 60], [80, 100, 120]]],
                [[[40, 50, 0], [30, 20, 10]]],
                [[[90, 80, 70], [10, 20, 30]]],
            ])
    out = torch.transpose(torch.mul(in2, torch.transpose(in1, 0, 3)), 0, 3)

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['transpose', torch.transpose(torch.mul(in2, torch.transpose(in1, 0, 3)), 0, 3), [in1]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


    domain_io = []
    domain_io.append(['mul', torch.transpose(torch.mul(in2, torch.transpose(in1, 0, 3)), 0, 3), [in2,torch.transpose(in1, 0, 3)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


    domain_io = []
    domain_io.append(['transpose', torch.transpose(torch.mul(in2, torch.transpose(in1, 0, 3)), 0, 3), [torch.mul(in2, torch.transpose(in1, 0, 3))]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

Composite49()

In [ ]:
# NOT SUPPORTED BY 16 APIs
def Composite50():
    if is16 == True:
        return
    in1 = torch.tensor(3)
    out_orig = torch.tensor([
                [0, 0, 0, 1, 0, 0],
                [0, 0, 0, 1, 0, 0],
                [0, 0, 0, 1, 0, 0],
                [0, 0, 0, 1, 0, 0],
                [0, 0, 0, 1, 0, 0],
            ])
    out = torch.nn.functional.one_hot(in1.expand(5), 6)

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.equal(out, out_orig))

    domain_io = []
    domain_io.append(['expand', torch.nn.functional.one_hot(in1, 6), [torch.tensor(3)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)

    domain_io = []
    domain_io.append(['one_hot', torch.nn.functional.one_hot(in1, 6), [in1, torch.tensor(6)]])
    X, Y = embed_tensor_for_model(domain_io)
    query_model(X, Y)


Composite50()

In [ ]:
print_stat()

In [ ]:
tc = 0
for t in sorted(total_correct):
    tc +=  total_correct[t]
    print(t, total_correct[t], tc/(total_benchmark-1))